<a href="https://colab.research.google.com/github/macbhaldar/Data-Science-Python-Handbook/blob/main/Association-Rules/Apriori_algorithm_online_retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Apriori Algorithm
##### Apriori Algorithm is a Machine Learning algorithm which is used to gain insight into the structured relationships between different items involved. 
##### The most prominent practical application of the algorithm is to recommend products based on the products already present in the user’s cart.

In [2]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
# Loading the Data
data = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# Exploring the columns of the data
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [5]:
# Exploring the different regions of transactions
data.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [6]:
# cleaning the data
# Stripping extra spaces in the description
data['Description'] = data['Description'].str.strip()

In [7]:
# Dropping the rows without any invoice number
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')

In [8]:
# Dropping all transactions which were done on credit
data = data[~data['InvoiceNo'].str.contains('C')]

### Splitting the data according to the region of transaction

In [9]:
# Transactions done in the United Kingdom
basket_UK = (data[data['Country'] =="United Kingdom"]
		.groupby(['InvoiceNo', 'Description'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('InvoiceNo'))

In [10]:
# Transactions done in France
basket_France = (data[data['Country'] =="France"]
		.groupby(['InvoiceNo', 'Description'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('InvoiceNo'))

In [16]:
# Transactions done in Germany
basket_Germany = (data[data['Country'] =="Germany"]
		.groupby(['InvoiceNo', 'Description'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('InvoiceNo'))

In [14]:
# Transactions done in Sweden
basket_Sweden = (data[data['Country'] =="Sweden"]
		.groupby(['InvoiceNo', 'Description'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('InvoiceNo'))

### Hot encoding the Data
##### Defining the hot encoding function to make the data suitable for the concerned libraries

In [17]:
def hot_encode(x):
	if(x<= 0):
		return 0
	if(x>= 1):
		return 1

# Encoding the datasets
basket_encoded = basket_UK.applymap(hot_encode)
basket_UK = basket_encoded

basket_encoded = basket_France.applymap(hot_encode)
basket_France = basket_encoded

basket_encoded = basket_Germany.applymap(hot_encode)
basket_Germany = basket_encoded

basket_encoded = basket_Sweden.applymap(hot_encode)
basket_Sweden = basket_encoded

### Building the models and analyzing the results

In [18]:
# Building the model for UK
frq_items = apriori(basket_UK, min_support = 0.01, use_colnames = True)
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                                       antecedents             consequents  \
116           (BEADED CRYSTAL HEART PINK ON STICK)        (DOTCOM POSTAGE)   
2019  (SUKI  SHOULDER BAG, JAM MAKING SET PRINTED)        (DOTCOM POSTAGE)   
2294         (HERB MARKER MINT, HERB MARKER THYME)  (HERB MARKER ROSEMARY)   
2302   (HERB MARKER PARSLEY, HERB MARKER ROSEMARY)     (HERB MARKER THYME)   
2300      (HERB MARKER PARSLEY, HERB MARKER THYME)  (HERB MARKER ROSEMARY)   

      antecedent support  consequent support   support  confidence       lift  \
116             0.011036            0.037928  0.010768    0.975728  25.725872   
2019            0.011625            0.037928  0.011196    0.963134  25.393807   
2294            0.010714            0.012375  0.010232    0.955000  77.173095   
2302            0.011089            0.012321  0.010553    0.951691  77.240055   
2300            0.011089            0.012375  0.010553    0.951691  76.905682   

      leverage  conviction  
116   0.010349 

In [19]:
# Building the model for France
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True)
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                                           antecedents  \
45                        (JUMBO BAG WOODLAND ANIMALS)   
259  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...   
270  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...   
300  (SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...   
302  (SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...   

                         consequents  antecedent support  consequent support  \
45                         (POSTAGE)            0.076531            0.765306   
259                        (POSTAGE)            0.051020            0.765306   
270                        (POSTAGE)            0.053571            0.765306   
300  (SET/6 RED SPOTTY PAPER PLATES)            0.102041            0.127551   
302    (SET/6 RED SPOTTY PAPER CUPS)            0.102041            0.137755   

      support  confidence      lift  leverage  conviction  
45   0.076531       1.000  1.306667  0.017961         inf  
259  0.051020       1.000  1.306667  0.011974     

In [20]:
# Building the model for Germany
frq_items = apriori(basket_Germany, min_support = 0.05, use_colnames = True)
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                                           antecedents consequents  \
35                         (PLASTERS IN TIN STRONGMAN)   (POSTAGE)   
51                   (RETROSPOT TEA SET CERAMIC 11 PC)   (POSTAGE)   
53                 (ROUND SNACK BOXES SET OF 4 FRUITS)   (POSTAGE)   
104  (ROUND SNACK BOXES SET OF4 WOODLAND, ROUND SNA...   (POSTAGE)   
33                          (PLASTERS IN TIN SPACEBOY)   (POSTAGE)   

     antecedent support  consequent support   support  confidence      lift  \
35             0.070022            0.818381  0.067834    0.968750  1.183740   
51             0.056893            0.818381  0.054705    0.961538  1.174928   
53             0.157549            0.818381  0.150985    0.958333  1.171012   
104            0.131291            0.818381  0.124726    0.950000  1.160829   
33             0.107221            0.818381  0.100656    0.938776  1.147113   

     leverage  conviction  
35   0.010529    5.811816  
51   0.008145    4.722101  
53   0.022049    4.3

In [21]:
# Building the model for Sweden
frq_items = apriori(basket_Sweden, min_support = 0.05, use_colnames = True)
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                        antecedents                        consequents  \
0     (12 PENCILS SMALL TUBE SKULL)      (PACK OF 72 SKULL CAKE CASES)   
1     (PACK OF 72 SKULL CAKE CASES)      (12 PENCILS SMALL TUBE SKULL)   
4           (36 DOILIES DOLLY GIRL)     (ASSORTED BOTTLE TOP  MAGNETS)   
5    (ASSORTED BOTTLE TOP  MAGNETS)            (36 DOILIES DOLLY GIRL)   
180  (CHILDRENS CUTLERY DOLLY GIRL)  (CHILDRENS CUTLERY CIRCUS PARADE)   

     antecedent support  consequent support   support  confidence  lift  \
0              0.055556            0.055556  0.055556         1.0  18.0   
1              0.055556            0.055556  0.055556         1.0  18.0   
4              0.055556            0.055556  0.055556         1.0  18.0   
5              0.055556            0.055556  0.055556         1.0  18.0   
180            0.055556            0.055556  0.055556         1.0  18.0   

     leverage  conviction  
0    0.052469         inf  
1    0.052469         inf  
4    0.052469       